<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Заразительность-зевоты" data-toc-modified-id="Заразительность-зевоты-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Заразительность зевоты</a></span></li><li><span><a href="#Тысячалетние-банкноты" data-toc-modified-id="Тысячалетние-банкноты-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Тысячалетние банкноты</a></span></li><li><span><a href="#GMAT-to-MBA" data-toc-modified-id="GMAT-to-MBA-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>GMAT to MBA</a></span><ul class="toc-item"><li><span><a href="#Part-1" data-toc-modified-id="Part-1-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Part 1</a></span></li><li><span><a href="#Part-2" data-toc-modified-id="Part-2-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Part 2</a></span></li></ul></li></ul></div>

# Заразительность зевоты

In [1]:
import scipy
import numpy as np
import pandas as pd
from statsmodels.stats.weightstats import *
from statsmodels.stats.proportion import proportion_confint

In [50]:
n1 = 34
pos1 = 10

n2 = 16
pos2 = 4

In [51]:
conf_interval_banner_a = proportion_confint(pos1, 
                                            n1,
                                            method = 'wilson')
conf_interval_banner_b = proportion_confint(pos2, 
                                            n2,
                                            method = 'wilson')

print('95%% confidence interval for a click probability, banner a: [%f, %f]' % conf_interval_banner_a)
print('95%% confidence interval for a click probability, banner b [%f, %f]' % conf_interval_banner_b)

95% confidence interval for a click probability, banner a: [0.168346, 0.461689]
95% confidence interval for a click probability, banner b [0.101821, 0.494983]


In [52]:
def proportions_diff_confint_ind(pos1, n1, pos2, n2, alpha = 0.05):    
    z = scipy.stats.norm.ppf(1 - alpha / 2.)
    
    p1 = pos1 / n1
    p2 = pos2 / n2
    
    left_boundary = (p1 - p2) - z * np.sqrt(p1 * (1 - p1)/ n1 + p2 * (1 - p2)/ n2)
    right_boundary = (p1 - p2) + z * np.sqrt(p1 * (1 - p1)/ n1 + p2 * (1 - p2)/ n2)
    
    return (left_boundary, right_boundary)

proportions_diff_confint_ind(pos1, n1, pos2, n2)

(-0.2175577216559601, 0.3057930157736072)

In [53]:
def proportions_diff_z_stat_ind(pos1, n1, pos2, n2):

    
    p1 = pos1 / n1
    p2 = pos2 / n2 
    P = float(p1*n1 + p2*n2) / (n1 + n2)
    
    return (p1 - p2) / np.sqrt(P * (1 - P) * (1. / n1 + 1. / n2))

z_stat = proportions_diff_z_stat_ind(pos1, n1, pos2, n2)
z_stat

0.32410186177608225

In [54]:
def proportions_diff_z_test(z_stat, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    if alternative == 'two-sided':
        return 2 * (1 - scipy.stats.norm.cdf(np.abs(z_stat)))
    
    if alternative == 'less':
        return scipy.stats.norm.cdf(z_stat)

    if alternative == 'greater':
        return 1 - scipy.stats.norm.cdf(z_stat)
    
pvalue = proportions_diff_z_test(z_stat, alternative='greater')

print("Pvalue:", pvalue)

Pvalue: 0.37293045872523534


# Тысячалетние банкноты

In [55]:
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [56]:
df = pd.read_table('banknotes.txt')
df.describe()

,X1,X2,X3,X4,X5,X6,real
count,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000
mean,214.896000,130.121500,129.956500,9.417500,10.650500,140.483500,0.500000
std,0.376554,0.361026,0.404072,1.444603,0.802947,1.152266,0.501255
min,213.800000,129.000000,129.000000,7.200000,7.700000,137.800000,0.000000
25%,214.600000,129.900000,129.700000,8.200000,10.100000,139.500000,0.000000
50%,214.900000,130.200000,130.000000,9.100000,10.600000,140.450000,0.500000
75%,215.100000,130.400000,130.225000,10.600000,11.200000,141.500000,1.000000
max,216.300000,131.000000,131.100000,12.700000,12.300000,142.400000,1.000000


In [57]:
y = df.real
X = df.drop('real', axis=1)

t_1 = ['X1', 'X2', 'X3']
t_2 = ['X4', 'X5', 'X6']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

In [58]:
model1 = LogisticRegression()
model1.fit(X_train[t_1], y_train)
y_pred1 = model1.predict(X_test[t_1])

model2 = LogisticRegression()
model2.fit(X_train[t_2], y_train)
y_pred2 = model2.predict(X_test[t_2])

In [78]:
accuracy_score(y_test, y_pred1), accuracy_score(y_test, y_pred2)

l1 = (y_test == y_pred1) * 1
l2 = (y_test == y_pred2) * 1

In [79]:
def proportions_diff_confint_rel(sample1, sample2, alpha = 0.05):
    z = scipy.stats.norm.ppf(1 - alpha / 2.)
    sample = list(zip(sample1, sample2))
    n = len(sample)
        
    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])
    
    left_boundary = float(f - g) / n  - z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    right_boundary = float(f - g) / n  + z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    return (left_boundary, right_boundary)

In [80]:
def proportions_diff_z_stat_rel(sample1, sample2):
    sample = list(zip(sample1, sample2))
    n = len(sample)
    
    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])
    
    return float(f - g) / np.sqrt(f + g - float((f - g)**2) / n )

In [81]:

print("Diff intervals: ", proportions_diff_confint_rel(l1, l2))

z_stat = proportions_diff_z_stat_rel(l1, l2)
print("Z statistic:", z_stat)

pvalue = proportions_diff_z_test(z_stat)

print("Pvalue:", pvalue)

Diff intervals:  (-0.3000547937203857, -0.059945206279614305)
Z statistic: -2.9386041680175268
Pvalue: 0.0032969384555543435


# GMAT to MBA

## Part 1

In [82]:
n1 = 200000
m1 = 525
std1 = 100

n2 = 100
m2 = 541.4

In [85]:
z_stat = (m2 - m1) / (std1 / np.sqrt(n2))
z_stat

1.6399999999999977

In [88]:
pvalue = proportions_diff_z_test(z_stat, 'greater')

print("Pvalue:", pvalue)

Pvalue: 0.05050258347410397


## Part 2

In [89]:
n1 = 200000
m1 = 525
std1 = 100

n2 = 100
m2 = 541.5

In [90]:
z_stat = (m2 - m1) / (std1 / np.sqrt(n2))
z_stat

1.65

In [91]:
pvalue = proportions_diff_z_test(z_stat, 'greater')

print("Pvalue:", pvalue)

Pvalue: 0.0494714680336481
